# Automatizarea intocmirii fisei pacientului

### Scop

Dezvoltarea unui sistem inteligent care să ajute medicii pentru a completa in mod automat fisa pacientului.

### Ideea de baza

Munca medicilor este plina de provocari. Mai ales cand trebuie sa faca multe task-uri, uneori simultan, precum realizarea si citirea unei ecografii si inregistrarea observatiilor facute. De aceea este nevoie de un sistem inteligent care sa transforme informatia audio inregistrata de catre un medic in format text si sa completeze in mod automat rubricile dedicate din fisa pacientului. Se va pleca de la inregistrari audio, se vor converti in format text si se va completa automat partea evidentiata cu galben din fisa pacientului (informatiile respective se vor salva intr-un tabel/jason si apoi se vor exporta intr-un document word)

---

### Descriere Formală (Matematică / Tehnică)

Datele de intrare:
𝑋 = { fișiere audio_i }
- Explicație: X reprezintă mulțimea fișierelor audio înregistrate de medic, fiecare corespunzând unei observații sau consultații.

Scopul aplicației:
𝐹: 𝑋 → 𝑌
- Explicație: F este funcția care transformă fișierele audio în fișe pacient completate.

Unde:
𝑌 = { fișe pacient completate }
- Explicație: Y este mulțimea fișelor completate cu informații structurate extrase din audio.

---

### Descompunerea Problemei

Problema poate fi descompusă în două sub-probleme:
##### 1. Speech-to-Text
𝑓₁ : audio → text
- Explicație: f1 folosește modele ASR (Romanian Wav2Vec2) pentru a transforma fișierele audio în text.
##### 2. Information Extraction
𝑓₂ : text → date structurate
- Explicație: f2 extrage rubricile relevante din text pentru completarea automată a fișei pacientului.

### Funcția Finală

Funcția finală este:
𝐹 = 𝑓₂ ∘ 𝑓₁
- Explicație: mai întâi transformăm audio-ul în text, apoi extragem datele structurate pentru completarea fișei pacientului.



In [19]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import shutil
import os

In [20]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline
import soundfile as sf
import torchaudio
import torch

model_name = "TransferRapid/whisper-large-v3-turbo_ro"

# Load processor and model
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def preprocess_audio(audio_path, processor):
    """Preprocess audio: load via soundfile, resample if needed, and convert to model input format."""
    # Încarcă audio cu dtype explicit float32
    waveform_np, sample_rate = sf.read(audio_path, dtype='float32')
    waveform = torch.from_numpy(waveform_np)

    # Convertește stereo → mono dacă e necesar
    if len(waveform.shape) > 1:
        waveform = waveform.mean(dim=0)  # Medie pe canale

    # Resample to 16kHz if needed
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)

    # Process audio into model input format (asigură numpy array float32)
    waveform_np = waveform.numpy() if isinstance(waveform, torch.Tensor) else waveform
    inputs = processor(waveform_np, sampling_rate=16000, return_tensors="pt")

    # Move inputs to device
    inputs = {key: val.to(device) for key, val in inputs.items()}

    return inputs

def transcribe(audio_path):
    """Generate transcription for an audio file."""
    inputs = preprocess_audio(audio_path, processor)

    forced_decoder_ids = processor.tokenizer.get_decoder_prompt_ids(language="romanian", task="transcribe")

    with torch.no_grad():
        generated_ids = model.generate(inputs["input_features"], forced_decoder_ids=forced_decoder_ids)

    transcription = processor.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    return transcription[0]

In [21]:
app = FastAPI()
UPLOAD_FOLDER = "uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

In [22]:
@app.post("/upload-audio/")
async def upload_audio(file: UploadFile = File(...)):
    file_path = os.path.join(UPLOAD_FOLDER, file.filename)
    content = await file.read()
    with open(file_path, "wb") as buffer:
        buffer.write(content)
    file_size = len(content)
    transcription = transcribe(file_path)
    return JSONResponse({
        "filename": file.filename,
        "size_bytes": file_size,
        "transcription": transcription
    })

In [23]:
import nest_asyncio
import uvicorn
import threading

nest_asyncio.apply()

def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

print("FastAPI server is running in the background on http://127.0.0.1:8000")


FastAPI server is running in the background on http://127.0.0.1:8000


INFO:     Started server process [407]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 48] error while attempting to bind on address ('127.0.0.1', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [24]:
# Manual/local test: pick first file from UPLOAD_FOLDER and transcribe
import os
files = os.listdir(UPLOAD_FOLDER)
if not files:
    print("No files found in uploads/ to transcribe.")
else:
    test_path = os.path.join(UPLOAD_FOLDER, files[0])
    print(f"Testing file: {test_path}")
    try:
        result = transcribe(test_path)
        print("Local transcription:", result)
    except Exception as e:
        print(f"Error transcribing {test_path}: {e}")


Task exception was never retrieved
future: <Task finished name='Task-67' coro=<Server.serve() done, defined at /Users/eduard/Development/MIRPR-Voice-To-Text/.venv1/lib/python3.10/site-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/Users/eduard/Development/MIRPR-Voice-To-Text/.venv1/lib/python3.10/site-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/asyncio/base_events.py", line 1519, in create_server
    raise OSError(err.errno, 'error while attempting '
OSError: [Errno 48] error while attempting to bind on address ('127.0.0.1', 8000): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/eduard/Devel

Testing file: uploads/test3.ogg
Local transcription: Aorta la inel, opt, aorta la sinusuri, doisprezece, aortă ascendentă, zece, a se treisprezece, vede, șase, vede, șase, vede, șase, vede, șase, vede, valva aortică, valva aortică, veice, treisprezece, bară șase, bară șase.


In [25]:
# Local-only transcription test (no HTTP endpoints)
local_audio_path = "/Users/eduard/Downloads/test3.ogg"  # adjust to your file
print(f"Loading and transcribing: {local_audio_path}")
try:
    transcript = transcribe(local_audio_path)
    print("Transcript:", transcript)
except Exception as e:
    print(f"Error during transcription: {e}")


Loading and transcribing: /Users/eduard/Downloads/test3.ogg
Transcript: Aorta la inel, opt, aorta la sinusuri, doisprezece, aortă ascendentă, zece, a se treisprezece, vede, șase, vede, șase, vede, șase, vede, șase, vede, valva aortică, valva aortică, veice, treisprezece, bară șase, bară șase.


In [26]:
# === EXTRACȚIE ENTITĂȚI MEDICALE CU MODEL NER GENERIC (PENTRU COMPARAȚIE) ===
import json
CALEA_MODEL_NER = "dumitrescustefan/bert-base-romanian-ner"

try:
    print("=" * 80)
    print("METODA 1: Model NER Generic (pentru referință)")
    print("=" * 80)
    print(f"\n⚠️  AVERTISMENT: Modelul {CALEA_MODEL_NER} NU este antrenat pe date medicale!")
    print("    Rezultatele vor fi sub-optime pentru termeni medicali.\n")

    # Încărcăm pipeline-ul NER generic
    print(f"Se încarcă modelul NER: {CALEA_MODEL_NER}...")
    ner_pipeline = pipeline(
        "ner",
        model=CALEA_MODEL_NER,
        tokenizer=CALEA_MODEL_NER,
        aggregation_strategy="simple"
    )
    print("✅ Model încărcat cu succes.")

    # Rulăm textul prin pipeline pentru a extrage entitățile
    entitati_extrase = ner_pipeline(transcript)

    print("\n--- Entități Extrase de NER Generic ---")
    for entitate in entitati_extrase:
        print(f"- {entitate['entity_group']}: {entitate['word']} (scor: {entitate['score']:.2f})")

    # Conversia în JSON Structurat
    fisa_pacient_json_generic = {}
    for entitate in entitati_extrase:
        tip_entitate = entitate['entity_group']
        valoare = entitate['word']
        if tip_entitate not in fisa_pacient_json_generic:
            fisa_pacient_json_generic[tip_entitate] = []
        fisa_pacient_json_generic[tip_entitate].append(valoare)

    print("\n--- JSON Structurat Generat (Model Generic) ---")
    json_output_string = json.dumps(fisa_pacient_json_generic, indent=4, ensure_ascii=False)
    print(json_output_string)

    # Salvare în fișier
    with open("fisa_pacient_output_generalist.json", "w", encoding="utf-8") as f:
        f.write(json_output_string)
    print(f"\n✅ Fișierul 'fisa_pacient_output_generalist.json' a fost salvat.")

except Exception as e:
    print(f"❌ A apărut o eroare la încărcarea modelului sau la procesare: {e}")


METODA 1: Model NER Generic (pentru referință)

⚠️  AVERTISMENT: Modelul dumitrescustefan/bert-base-romanian-ner NU este antrenat pe date medicale!
    Rezultatele vor fi sub-optime pentru termeni medicali.

Se încarcă modelul NER: dumitrescustefan/bert-base-romanian-ner...


Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Model încărcat cu succes.

--- Entități Extrase de NER Generic ---
- LABEL_0: Aorta la inel, (scor: 0.99)
- LABEL_25: opt (scor: 1.00)
- LABEL_0: , aorta la sinusuri, (scor: 1.00)
- LABEL_25: doisprezece (scor: 1.00)
- LABEL_0: , aortă ascendentă, (scor: 1.00)
- LABEL_25: zece (scor: 1.00)
- LABEL_0: , a se (scor: 1.00)
- LABEL_25: treisprezece (scor: 1.00)
- LABEL_0: , vede, (scor: 1.00)
- LABEL_25: șase (scor: 1.00)
- LABEL_0: , vede, (scor: 1.00)
- LABEL_25: șase (scor: 1.00)
- LABEL_0: , vede, (scor: 1.00)
- LABEL_25: șase (scor: 1.00)
- LABEL_0: , vede, (scor: 1.00)
- LABEL_25: șase (scor: 1.00)
- LABEL_0: , vede, valva aortică, valva aortică, veice, (scor: 1.00)
- LABEL_25: treisprezece (scor: 1.00)
- LABEL_0: , bară (scor: 0.99)
- LABEL_25: șase (scor: 1.00)
- LABEL_0: , bară (scor: 0.99)
- LABEL_25: șase (scor: 0.99)
- LABEL_0: . (scor: 1.00)

--- JSON Structurat Generat (Model Generic) ---
{
    "LABEL_0": [
        "Aorta la inel,",
        ", aorta la sinusuri,",
        "

In [27]:
# === EXTRACȚIE ENTITĂȚI MEDICALE CU EXTRACTOR SPECIALIZAT (RECOMANDAT) ===
from medical_entity_extractor import MedicalEntityExtractor, process_medical_transcription

print("\n" + "=" * 80)
print("METODA 2: Extractor Medical Specializat (RECOMANDAT)")
print("=" * 80)
print("✅ Folosește pattern matching și reguli specifice domeniului medical\n")

# Inițializează extractorul
extractor = MedicalEntityExtractor()

# Extrage toate entitățile medicale
fisa_pacient = extractor.extract_all_entities(transcript)

# Afișează rezultatele
print("\n📋 MĂSURĂTORI ECOGRAFICE EXTRASE:")
print("-" * 80)
if fisa_pacient.masuratori_ecografice:
    for i, masurare in enumerate(fisa_pacient.masuratori_ecografice, 1):
        print(f"{i}. {masurare['structura_anatomica']}: {masurare['valoare_numerica']} {masurare['unitate_masura']}")
else:
    print("   Nicio măsurătoare detectată")

print("\n💊 MEDICAMENTE EXTRASE:")
print("-" * 80)
if fisa_pacient.medicamente:
    for med in fisa_pacient.medicamente:
        print(f"   • {med['nume']} - {med['dozaj']} ({med['frecventa']})")
else:
    print("   Niciun medicament detectat")

print("\n🩺 SIMPTOME EXTRASE:")
print("-" * 80)
if fisa_pacient.simptome:
    for simptom in fisa_pacient.simptome:
        print(f"   • {simptom}")
else:
    print("   Niciun simptom detectat")

print("\n🔍 DIAGNOSTICE EXTRASE:")
print("-" * 80)
if fisa_pacient.diagnostice:
    for diagnostic in fisa_pacient.diagnostice:
        print(f"   • {diagnostic}")
else:
    print("   Niciun diagnostic detectat")

# Salvează în format JSON standard
print("\n" + "=" * 80)
print("SALVARE DATE STRUCTURATE")
print("=" * 80)

extractor.save_to_json(fisa_pacient, "fisa_pacient_medical_structured.json")

# Generează și salvează format FHIR
fhir_observations = extractor.to_fhir_observation(fisa_pacient.masuratori_ecografice)
with open("fhir_observations.json", "w", encoding="utf-8") as f:
    json.dump(fhir_observations, f, indent=4, ensure_ascii=False)
print(f"✅ Observații FHIR salvate în: fhir_observations.json")

# Afișează JSON-ul complet
print("\n" + "=" * 80)
print("JSON COMPLET (cu format FHIR)")
print("=" * 80)
print(extractor.to_json(fisa_pacient))

print("\n" + "=" * 80)
print("✅ PROCESARE COMPLETĂ!")
print("=" * 80)
print("\n📊 FIȘIERE GENERATE:")
print("   1. fisa_pacient_medical_structured.json  - Date structurate complete")
print("   2. fhir_observations.json                - Format FHIR pentru integrare")
print("   3. fisa_pacient_output_generalist.json   - Rezultat model NER generic (pentru comparație)")
print("\n💡 TIP: Folosește 'fisa_pacient_medical_structured.json' pentru generarea raportului Word!")



METODA 2: Extractor Medical Specializat (RECOMANDAT)
✅ Folosește pattern matching și reguli specifice domeniului medical


📋 MĂSURĂTORI ECOGRAFICE EXTRASE:
--------------------------------------------------------------------------------
1. aorta la inel: 8.0 mm
2. aorta la sinusuri: 12.0 mm
3. aortă ascendentă: 10.0 mm

💊 MEDICAMENTE EXTRASE:
--------------------------------------------------------------------------------
   Niciun medicament detectat

🩺 SIMPTOME EXTRASE:
--------------------------------------------------------------------------------
   Niciun simptom detectat

🔍 DIAGNOSTICE EXTRASE:
--------------------------------------------------------------------------------
   Niciun diagnostic detectat

SALVARE DATE STRUCTURATE
✅ Fișa pacientului salvată în: fisa_pacient_medical_structured.json
✅ Observații FHIR salvate în: fhir_observations.json

JSON COMPLET (cu format FHIR)
{
    "masuratori_ecografice": [
        {
            "structura_anatomica": "aorta la inel",
      

In [28]:
# === GENERARE RAPORT WORD (CONFORM SECȚIUNII 5.2 DIN GHID) ===
from word_report_generator import MedicalReportGenerator, generate_word_report
from datetime import datetime

print("\n" + "=" * 80)
print("GENERARE RAPORT WORD AUTOMAT")
print("=" * 80)

# Metoda 1: Raport simplu (fără șablon)
print("\n📄 Generare raport simplu...")
try:
    raport_path = generate_word_report(
        json_path="fisa_pacient_medical_structured.json",
        output_path=f"raport_medical_{datetime.now().strftime('%Y%m%d_%H%M%S')}.docx",
        use_template=False
    )
    print(f"✅ Raport generat: {raport_path}")
except Exception as e:
    print(f"❌ Eroare la generarea raportului: {e}")

# Metoda 2: Raport cu șablon personalizat
print("\n📝 Generare șablon Word personalizabil...")
generator = MedicalReportGenerator()
try:
    generator.create_template("template_fisa_pacient.docx")

    # Generează raport folosind șablonul
    print("\n📄 Generare raport din șablon...")
    raport_template_path = generate_word_report(
        json_path="fisa_pacient_medical_structured.json",
        output_path=f"raport_medical_template_{datetime.now().strftime('%Y%m%d_%H%M%S')}.docx",
        use_template=True,
        template_path="template_fisa_pacient.docx"
    )
    print(f"✅ Raport din șablon generat: {raport_template_path}")
except Exception as e:
    print(f"❌ Eroare la generarea raportului cu șablon: {e}")

print("\n" + "=" * 80)
print("🎉 PIPELINE COMPLET FINALIZAT!")
print("=" * 80)
print("\n📋 REZULTATE FINALE:")
print("   ✅ Audio → Text (ASR cu Whisper)")
print("   ✅ Text → Entități Structurate (Pattern Matching Medical)")
print("   ✅ Entități → JSON (cu format FHIR)")
print("   ✅ JSON → Raport Word (Formatat și Lizibil)")
print("\n💡 URMĂTORII PAȘI:")
print("   1. Editează template_fisa_pacient.docx în Word pentru personalizare")
print("   2. Fine-tuning model NER pentru domeniul medical (opțional, pentru acuratețe maximă)")
print("   3. Integrează cu sistemul medical existent (FHIR, HL7)")



GENERARE RAPORT WORD AUTOMAT

📄 Generare raport simplu...
✅ Raport Word generat cu succes: raport_medical_20251024_195754.docx
✅ Raport generat: raport_medical_20251024_195754.docx

📝 Generare șablon Word personalizabil...
✅ Șablon creat: template_fisa_pacient.docx
💡 Poți edita acest șablon în Word și apoi îl poți folosi cu create_report_with_template()

📄 Generare raport din șablon...
✅ Raport Word generat din șablon: raport_medical_template_20251024_195754.docx
✅ Raport din șablon generat: raport_medical_template_20251024_195754.docx

🎉 PIPELINE COMPLET FINALIZAT!

📋 REZULTATE FINALE:
   ✅ Audio → Text (ASR cu Whisper)
   ✅ Text → Entități Structurate (Pattern Matching Medical)
   ✅ Entități → JSON (cu format FHIR)
   ✅ JSON → Raport Word (Formatat și Lizibil)

💡 URMĂTORII PAȘI:
   1. Editează template_fisa_pacient.docx în Word pentru personalizare
   2. Fine-tuning model NER pentru domeniul medical (opțional, pentru acuratețe maximă)
   3. Integrează cu sistemul medical existent (F